In [ ]:
menor = str(datetime.datetime.today() - datetime.timedelta(days=90))[:7]

mayor = str(datetime.datetime.today() - datetime.timedelta(days=70))[:7]

In [18]:
mayor

'2020-03'

datetime.datetime(2020, 5, 15, 0, 0)

In [32]:
menor = str(datetime.datetime.strptime("15-05-2020", "%d-%m-%Y") - datetime.timedelta(days=70))[:7]

In [33]:
mayor

'2020-02'

In [34]:
menor

'2020-03'

In [1]:
from dbconn import querydb, querydbtopandas 
import numpy as np
import pandas as pd
import datetime
import os

In [2]:
prestadores_modelo = pd.read_excel("prestadores_modelo.xlsx",usecols= ["id_pre_prestador"])["id_pre_prestador"].tolist()
modelos = str(prestadores_modelo).strip("[").strip("]")

In [3]:
prestaciones_excluidas = pd.read_excel("prestaciones_excluidas.xlsx",usecols = ["prestac_pres_prestacion"], dtype=str )["prestac_pres_prestacion"].tolist()
prestaciones_excluidas = str(prestaciones_excluidas).strip("[").strip("]")

In [4]:
provincias_zonas = pd.read_csv("provincias_zonas.csv")

In [5]:
provincias_zonas.codigo_zona.unique()

array([  1,   2,   3,   4,   5, 100])

In [6]:
provincias_zonas.columns

Index(['codigo_zona', 'zona', 'provincia', 'codigo_provincia '], dtype='object')

In [7]:
provincias_zonas["codigo_provincia "].unique()

array([ 1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 17, 18, 19,
       20, 21, 22, 23, 24, 25,  2, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 68])

In [8]:
# clasificacion incorrecta del nomenclador
# no oncluyen cientro prestadores
zona = "2,3,4,5"
provincia = "1,2"

In [9]:
mayor= "2020-03"
menor = "2019-02"

In [10]:
data = querydbtopandas(f"""select a11.id_pres_prestacion,
       a16.prestac_pres_prestacion,
       a11.id_pre_prestador,
       a13.desc_pre_nombre,
       a11.id_afi_afiliado ,  
       Q.deno,
       a11.id_tie_dia_consumo,   
       FLOOR(DATEDIFF(year, a14.naci_fecha, a11.id_tie_dia_consumo)) edad_afiliado 

from        DBA.ft_ordenes        a11
       join        DBA.d_pre_prestador        a13
         on        (a11.id_pre_prestador = a13.id_pre_prestador)
       join        DBA.afi_afiliados        a14
         on        (a11.id_afi_afiliado = a14.id_afiliado)
       join        DBA.d_pres_prestacion        a16
         on        (a11.id_pres_prestacion = a16.id_pres_prestacion)     
       join       DBA.d_prof_convertir_especialidad a19  
        on         (a13.id_pre_especialidad  = a19.id_prof_especialidad)
       LEFT JOIN dba.prestad_costo_centros E
        ON a11.id_pre_prestador = E.prestad and E.baja_fecha is null and e.prepaga = ( select min(PCC1.prepaga) from dba.prestad_costo_centros PCC1 where PCC1.prestad =  e.prestad )   
       LEFT JOIN dba.costo_centros Q
        ON E.centro = Q.centro
where        (a13.id_pre_tipo in ('E', 'P')            
and a11.id_nom_nomenclador in (5)            
and cast(id_tie_dia_consumo as date) >= date("{menor}") and cast(id_tie_dia_consumo as date) < date("{mayor}")
and a13.id_afi_zona in ({zona})
and a13.id_afi_provincia in ({provincia})

and RTRIM(a16.prestac_pres_prestacion) not in ({prestaciones_excluidas}))
group by a11.id_pres_prestacion,
       a16.prestac_pres_prestacion,
       a11.id_pre_prestador,
       a13.desc_pre_nombre,
       Q.deno,
       a11.id_afi_afiliado,
       a11.id_tie_dia_consumo,
       edad_afiliado""")

In [11]:
data.id_tie_dia_consumo = data.id_tie_dia_consumo.astype(str).str[:7]

In [12]:
data.id_tie_dia_consumo.nunique()

13

In [13]:
for i in data.id_tie_dia_consumo.unique():
    df = data[data["id_tie_dia_consumo"] == i]
        
    #filtro con topes minimos para prestadores
    #total_consultas_prestador = df["id_pre_prestador"].value_counts()
    #mas_de_30_consultas = total_consultas_prestador[total_consultas_prestador >= 30].index
    #df = df[df["id_pre_prestador"].isin(mas_de_30_consultas)]
    
    reconsultas = df.groupby(["id_pre_prestador", "desc_pre_nombre", "deno", "id_afi_afiliado"])["id_afi_afiliado"].count().rename("CANTIDADORDENES0").reset_index()
    reconsultas["uno"] = 1

    pivot_1 = pd.pivot_table(reconsultas, values='uno', index=['id_pre_prestador', "desc_pre_nombre", "deno"],
                 columns=['CANTIDADORDENES0'], aggfunc=np.sum).fillna(0)

    pivot_1["Más de 4 - Q afiliados"] = pivot_1[pivot_1.columns.tolist()[4:]].sum(axis=1)

    edad_promedio_afiliados = pd.DataFrame(df.groupby(["id_pre_prestador",  "desc_pre_nombre", "deno"])["edad_afiliado"].mean())
    total_consultas_prestador = reconsultas.groupby(["id_pre_prestador",  "desc_pre_nombre", "deno"])["CANTIDADORDENES0"].sum()
    pivot = pd.concat([total_consultas_prestador,edad_promedio_afiliados, pivot_1],axis=1)
    pivot.reset_index(inplace=True)
    if 1 not in pivot.columns:
        pivot[1] = 0
    if 2 not in pivot.columns:
        pivot[2] = 0
    if 3 not in pivot.columns:
        pivot[3] = 0
    if 4 not in pivot.columns:
        pivot[4] = 0

    pivot["Consultas - participación %"] =  (pivot[1]/pivot["CANTIDADORDENES0"])*100
    pivot["2 consultas - participación %"] =  ((pivot[2]*2)/pivot["CANTIDADORDENES0"])*100
    pivot["3 consultas - participación %"] =  ((pivot[3]*3)/pivot["CANTIDADORDENES0"])*100

    pivot["4 consultas - participación %"] =  ((pivot[4]*4)/pivot["CANTIDADORDENES0"])*100

    pivot["Participación %"] = 100 - pivot["Consultas - participación %"]
    pivot["Más de 4 - participación %"] = pivot["Participación %"] - (pivot["2 consultas - participación %"]+pivot["3 consultas - participación %"]+pivot["4 consultas - participación %"])
    pivot["Total de reconsultas"] = pivot[2] + pivot[3] + pivot[4] + pivot["Más de 4 - Q afiliados"]
    pivot.rename(columns= {1: "Consultas - Q afiliados", 2: "2 consultas - Q afiliados", 3: "3 consultas - Q afiliados", 4: "4 consultas - Q afiliados", "desc_pre_nombre": "Razon social",
    "deno": "Especialidad","id_pre_prestador": "Prestador", "edad_afiliado": "Promedio edad de afiliados", "CANTIDADORDENES0": "Total Consultas" },inplace=True)    
    pivot = pivot.sort_values("Participación %", ascending=False).round(2)
    pivot = pivot.reset_index(drop=True)
    pivot = pivot[['Prestador', 'Razon social', 'Especialidad', 'Total Consultas',
           'Promedio edad de afiliados', 'Consultas - Q afiliados','Consultas - participación %',
           '2 consultas - Q afiliados', '2 consultas - participación %',
           '3 consultas - Q afiliados', '3 consultas - participación %',
           '4 consultas - Q afiliados',  '4 consultas - participación %',
           'Más de 4 - Q afiliados',  'Más de 4 - participación %',
           'Total de reconsultas','Participación %']]
    
    pivot.to_parquet(f'datos_consultas/{i}.gzip',compression='gzip') 

In [53]:
pivot

,Prestador,Razon social,Especialidad,Total Consultas,Promedio edad de afiliados,Consultas - Q afiliados,Consultas - participación %,2 consultas - Q afiliados,2 consultas - participación %,3 consultas - Q afiliados,3 consultas - participación %,4 consultas - Q afiliados,4 consultas - participación %,Más de 4 - Q afiliados,Más de 4 - participación %,Total de reconsultas,Participación %
0,172627,"Ferro,Francisco E.",Gastroenterología Pediátrica,34,48.53,0.0,0.00,11.0,64.71,0.0,0.00,3.0,35.29,0.0,0.00,14.0,100.00
1,117056,"Abeszyc,Daniel A.",Ginecología,54,51.94,1.0,1.85,25.0,92.59,1.0,5.56,0.0,0.00,0.0,0.00,26.0,98.15
2,56951,"Koll,Norma E.",Dermatología,82,39.02,6.0,7.32,31.0,75.61,0.0,0.00,2.0,9.76,1.0,7.32,34.0,92.68
3,165835,"Guarrera,Gladys L.",Pediatría,65,40.14,8.0,12.31,9.0,27.69,5.0,23.08,6.0,36.92,0.0,-0.00,20.0,87.69
4,136032,"Bilbao Curutchet,Pablo",Pediatría,88,8.45,14.0,15.91,18.0,40.91,10.0,34.09,2.0,9.09,0.0,0.00,30.0,84.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1452,70177,"Martínez,Jorge A.",Cardiología,215,43.76,215.0,100.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00
1453,210809,"Bermejo,Irene G.",Dermatología,54,34.02,54.0,100.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00
1454,24344,"Manzitti,Julio",Oftalmología,41,16.37,41.0,100.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00
1455,141087,"Sánchez Voci,Christian P.",Cardiología Pediátrica,68,9.06,68.0,100.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00


In [3]:
fechas = [i for i in os.listdir('datos_consultas/') if len(i) == 12]
completo = pd.DataFrame()
for i in fechas:
    completo = pd.concat([completo, pd.read_parquet(f'datos_consultas/{i}')])

In [4]:

completo.Prestador.nunique()

2108

In [7]:
pd.Series(completo.Prestador.unique()).to_csv("prestadores_unicos_consultas.csv")